# Mutli-Dimensional Trajecoty Kalman Filtering. 
Tutorial by : Mahmoud Ouf. | 
arch.mahmoud.ouf111[at]gmail.com | 
[GitHub](https://github.com/MahmoudAbdelRahman)

# 1.Basic flow :
![](im/kalmanFilter_illustration.png)
Figure adapted from online tutorial : https://www.youtube.com/watch?v=Fuy73n6_bBc&list=PLX2gX-ftPVXU3oUFNATxGXY90AULiqnWT&index=27


In [276]:
#$P_0$ is the Process error in the Process covrariance matrix, the initial value of this matrix is first assumed. then it its finetuned each iteration.

#### Import the important libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import math
import pandas as pd

# 2. Intital conditions and datasources. 
First lets load the dataset of a moving [Projectile](https://en.wikipedia.org/wiki/Projectile_motion) object. 
Source files are located in `.\df.csv`, it consists of the following data : 
`'X_groundTruth', 'Y_groundTruth', 'X_measured', 'Y_measured', 'Vx','Vy', 'X_error', 'Y_error'` as shown in the following figure. 

![](im/01_Projectile.gif)

In [2]:
data = pd.read_csv("./df.csv")
print(data.columns)
data.head()

Index(['X_groundTruth', 'Y_groundTruth', 'X_measured', 'Y_measured', 'Vx',
       'Vy', 'X_error', 'Y_error'],
      dtype='object')


,X_groundTruth,Y_groundTruth,X_measured,Y_measured,Vx,Vy,X_error,Y_error
0,2.0,0.000,2.000000,0.000000,5.0,50.00,0.000000,0.000000
1,2.0,0.000,2.410879,0.366290,5.0,50.00,0.410879,0.366290
2,2.5,4.951,4.936054,4.031234,5.0,49.02,2.436054,-0.919766
3,3.0,9.804,3.694652,10.302644,5.0,48.04,0.694652,0.498644
4,3.5,14.559,1.515819,14.621272,5.0,47.06,-1.984181,0.062272


In [27]:
ff = data.Vy.tolist()
for i in range(3,20):
    print("acceleration :",round((ff[i]-ff[i-1])/0.1,3))

acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8
acceleration : -9.8


# 3. Kalman Filter for 2D Trajectory estimation. 
### Following the flow chart indicated in 1 The workflow will be divided into 8 different secitons. 
+ ### 3.0 Initialize data. 
+ ### 3.1 State Matrix $X_k$
+ ### 3.2 Initial process covariance matrix $P_{k_i}$
+ ### 3.3 Process to the covariance matrix $P_{k_p}$
+ ### 3.4 Kalman Gain $k$
+ ### 3.5 Current observation $Y$
+ ### 3.6 Update current state matrix based on Kalman Gain $X_k$
+ ### 3.7 Update the current process covariance matrix $P_k$
+ ### 3.8 Process to the next iteration.


---
## 3.0 First, we intialize data 

In [3]:
x_0 = 50.
y_0 = 0.0

v_x0 = 5.0
v_y0 = 50.0

delta_t = 0.1 # sec

acc_x = 0.0
acc_y = -9.8

error_x = 10.0
error_y = 10.0

error_vx = 5.0
error_vy = 10.0

## 3.1. Then, we define the State Matrix 
## $$ X_{k_p}  = A X_{k-1} + B U_{k} + W_k $$
Where $X$ is the state matrix, $k$ represents the time iteration, $U$ is the acceleration matrix, $A$ and $B$ are adjustment matrices. 
#### - In case of 2D matrix : 
#### $$ A = \begin{vmatrix} 
1 & 0 & \Delta T & 0 \\
0 & 1 & 0 & \Delta T \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 
\end{vmatrix}, X = \begin{vmatrix} 
X_{pos} \\
Y_{pos} \\
X_{vel} \\
Y_{vel} 
\end{vmatrix} $$

While
$$ B = \begin{vmatrix} 
\frac{1}{2} \Delta t^2 & 0\\
0 & \frac{1}{2} \Delta t^2  \\ 
\Delta t  & 0  \\ 
a_{y} & \Delta t  \\ 
\end{vmatrix},U = \begin{vmatrix} 
a_{x} \\
a_{y} \\ 
\end{vmatrix} $$

#### - In case of 3D matrix : 
#### $$ A = \begin{vmatrix} 
1 & 0 & 0 & \Delta T & 0 & 0 \\
0 & 1 & 0 & 0 & \Delta T & 0 \\
0 & 0 & 1 & 0 & 0 & \Delta T \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 1
\end{vmatrix}, X = \begin{vmatrix} 
X_{pos} \\
Y_{pos} \\
Z_{pos} \\
X_{vel} \\
Y_{vel}\\
Z_{vel} 
\end{vmatrix} $$

In this tutorial, we only consider 2d problem. 

In [14]:
def X_kp(X, U, W = None, delta_t = 1.0):
    """
    X_kp is the predicted state matrix, 
    @param : X : is the numpy arry of shape (4, 1) contains the initial [[xpos], [ypos], [xvel], [yvel]]
    @param : U : is the acceleration matrix, of shape (4, 4)
    @param : W : is the error matrix.
    @param : delta_t : is the time step. 
    
    return : Current initial state matrix. 
    """
    A = np.identity(4)
    A[0, 2] = delta_t
    A[1, 3] = delta_t
    
    B = np.zeros((4, 2))
    B[0,0] = 0.5*delta_t*delta_t    
    B[1,1] = 0.5*delta_t*delta_t      
    B[2,0] = delta_t
    B[3,1] = delta_t
    
    AX = np.dot(A, X)
    BU = np.dot(B, U)
    
    if W == None:
        W = np.zeros(4,1)
        
    return AX + BU + W


## 3.2 Initial process covariance matrix $P_{{k}}$
This initial declaration of the process covariance matrix only occurs once. Then this step will be passed.
## $$ P_{k} = A P_{k-1}A^T + Q_k $$
$A$ is an adjustement matrix similar to that in the previous equation 3.1, while $A^T$ is its transpose. $Q_k$ is error resource.
$$ A = \begin{vmatrix} 
1 & 0 & \Delta T & 0 \\
0 & 1 & 0 & \Delta T \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 
\end{vmatrix}, A^T = \begin{vmatrix} 
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
\Delta T & 0 & 1 & 0 \\
0 & \Delta T & 0 & 1 
\end{vmatrix}$$ 

Covariance Matrix : $$ P_{k-1} = \begin{vmatrix} 
{X_{k-1}}^2        & X_{k-1}.Y_{k-1}     & X_{k-1}.V_{X_{k-1}} & X_{k-1}.V_{Y_{k-1}} \\
X_{k-1}.Y_{k-1}    & {Y_{k-1}}^2         & Y_{k-1}.V_{X_{k-1}} & Y_{k-1}.V_{Y_{k-1}} \\
X_{k-1}.V_{X_{k-1}}& Y_{k-1}.V_{X_{k-1}} & {V_{X_{k-1}}}^2     & V_{X_{k-1}}.V_{Y_{k-1}} \\
X_{k-1}.V_{Y_{k-1}}& Y_{k-1}.V_{Y_{k-1}} & V_{X_{k-1}}.V_{Y_{k-1}} & {V_{Y_{k-1}}}^2 
\end{vmatrix}$$ 

In [ ]:
# initial conditions
x_0 = 200. #m --> the initial x position
y_0 = 500. #m --> the initial y position

v_0x = 50. #m/s --> the initial x velocity. 
v_0y = 20. #m/s --> the initial y velocity

#initial conditions.
#    Acceleration. 
a_x = 2.   #m/sec^2
a_y = 2.   #m/sec^2

#Delta time --> 1 sec
d_t = 1.   #sec

#The observation error.
#    Velocity error. 
v_x = 10.  #m/s
v_y = 10. #m/s

#
dx = 10.  #m
dy = 10.  #m 

In [35]:
aa = np.array([[2,4,5,2]]).T
dd = np.cov(aa)
dd

/home/mahmoud/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Degrees of freedom <= 0 for slice
  


array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])

In [271]:
"""### 2. Process Covariance Matrix
#### $$ P_{k_P} = A P_{k-1}A^T + Q_k $$
### 3. Kalman Gain
#### $$K = \frac{P_{k_p}H^T}{H P_{k_p} H^T + R}$$
$K$ is the Kalman gain, $H$ is an adjusting matrix and $R$ is the observation error (usually caused by sensor noise).
#### $$X_k = X_{k_p} + K [Y-H X_k] $$
$X_k$ in this case is the new calculated state matrix after calculating Kalman Gain. 
#### $$ P_k = (I - K H)P_{k_p} $$
### 4. Sensor Noise Covariance Matrix
"""

'### 2. Process Covariance Matrix\n#### $$ P_{k_P} = A P_{k-1}A^T + Q_k $$\n### 3. Kalman Gain\n#### $$K = \x0crac{P_{k_p}H^T}{H P_{k_p} H^T + R}$$\n$K$ is the Kalman gain, $H$ is an adjusting matrix and $R$ is the observation error (usually caused by sensor noise).\n#### $$X_k = X_{k_p} + K [Y-H X_k] $$\n$X_k$ in this case is the new calculated state matrix after calculating Kalman Gain. \n#### $$ P_k = (I - K H)P_{k_p} $$\n### 4. Sensor Noise Covariance Matrix\n'